# IESB - Graduacao - CIA028 - Costa Rica

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

# I - Análise Exploratória e tratamento dos dados


In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
df_all.columns

In [ ]:
df['Target'].isnull().sum()

In [ ]:
df_all['Target'].value_counts()

In [ ]:
# Verificando tamanhos e tipos
df_all.info()

In [ ]:
df.dtypes.value_counts()

In [ ]:
df_all.select_dtypes(include='object')


In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
mapeamento = {'yes': 1, 'no': 0}
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Visualizando do comando info
df_all.info()

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

In [ ]:
 # Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Qual a cara dos dados de v18q
df_all['v18q'].value_counts()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned, meaneduc e rez_esc
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
df_all.describe()


In [ ]:
#Visualização da distribuição das variáveis
df_all.hist(figsize=(20,15))
plt.show()

# II - Aplicando o Modelo Random Forest


In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Instanciando o random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Verificando a classe target nos dados de treino
train['Target'].value_counts(normalize=True)

In [ ]:
# Limitando o treinamento as/aos chefas/es de familia

# Coluna parentesco1
heads = train[train['parentesco1'] == 1]

In [ ]:
# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF2
# Dados de treinao apenas dos chefes/chefas de familia

rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

rf2.fit(heads[feats], heads['Target'])

test['Target'] = rf2.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Qual o tamanho da base de treino heads?
heads.shape

In [ ]:
# Verificando os valores da coluna hhsize
train['hhsize'].value_counts()

In [ ]:
# Verificando os dados da coluna tamviv
train['tamviv'].value_counts()

In [ ]:
# Verificando os dados da coluna tamhog
train['tamhog'].value_counts()

In [ ]:
# Feature Engineering / Criação de novas colunas

# Relação tamanho da casa / moradores
df_all['hhsize-pc'] = df_all['hhsize'] / df_all['tamviv']

# Relação qtde celulares / moradores
df_all['mobile-pc'] = df_all['qmobilephone'] / df_all['tamviv']

# Relaçao qtde de tablets / moradores
df_all['tablet-pc'] = df_all['v18q1'] / df_all['tamviv']

# Relação qtde de quartos / moradores
df_all['rooms-pc'] = df_all['rooms'] / df_all['tamviv']

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF3
# Dados de treino com 4 colunas a mais

rf3 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

rf3.fit(train[feats], train['Target'])

test['Target'] = rf3.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf3.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Juntando as abordagens

# Selecionando para treio só parentesco1 == 1
heads2 = train[train['parentesco1'] == 1]

In [ ]:
# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF4
# Dados de treino apenas dos chefes/chefas de familia e 4 colunas a mais

rf4 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

rf4.fit(heads2[feats], heads2['Target'])

test['Target'] = rf4.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Verificando os dados de treino e teste
train.shape, test.shape

In [ ]:
# Usando o Out-Of-Bag (oob_score)

# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF3
# Dados de treino com 4 colunas a mais + oob_score

rf5 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42, oob_score=True, class_weight='balanced')

rf5.fit(train[feats], train['Target'])

test['Target'] = rf5.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)


In [ ]:
# Voltando as origens

# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Preenchendo os valores nulos
df_all['v2a1'].fillna(-1, inplace=True)
df_all['v18q1'].fillna(0, inplace=True)
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'edjefe', 'edjefa','Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Usando o Out-Of-Bag (oob_score)

# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF6
# oob_score + class_weight

rf6 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42, oob_score=True, class_weight='balanced')

rf6.fit(train[feats], train['Target'])

test['Target'] = rf6.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Copiando do campeão
rf7 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

rf7.fit(train[feats], train['Target'])

test['Target'] = rf7.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

# III - Modificações no Modelo (Prova 1)

Existe uma diferença entre propriedades de cada uma das árvores, e as propriedades da floresta (sacola de árvores).
Os principais parâmetros são n_estimators (número de árvores na floresta) e max_features (o tamanho do subconjunto de variáveis que serão consideradas quando o modelo faz a divisão do nó).
A documentação do scikit-learn recomenda utilizar max_features="sqrt" para tarefas de classificação. Uma possibilidade seria não utilizar um subconjunto, e sim todas as variáveis, como é recomendado por Geurts, Erns e Wehenkel (2006) para o modelo the RandomForestRegressor

https://orbi.uliege.be/bitstream/2268/9357/1/geurts-mlj-advance.pdf


In [ ]:
# Mudança 1 - max_features=n_features
rf8 = RandomForestClassifier(max_depth=None, max_features=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

rf8.fit(train[feats], train['Target'])

test['Target'] = rf8.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

O modelo RandomForestClassifier pode utilizar dois métodos de avaliação da qualidade da divisão: gini e entropia.
Gini - Mede o quanto um elemento aleatoriamente escolhido do conjunto seria incorretamente classificado. Quanto maior o valor gini, mais "impuro", ou seja, mais o elemento aleatório seria incorretamente classificado.
Entropia - mede a "desordem" geral de uma segmentação da árvore. É um método mais pesado computacionalmente do que o gini, por usar logbase2 das probabilidades

Laura Elena Raileanu e Kilian Stoffel estimam que só há diferença entre as duas medidas em 2% dos casos.

https://www.unine.ch/files/live/sites/imi/files/shared/documents/papers/Gini_index_fulltext.pdf


In [ ]:
# Mudança 2 - criterion="entropy"
rf9 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2, criterion="entropy",
                            verbose=0, class_weight='balanced')

rf9.fit(train[feats], train['Target'])

test['Target'] = rf9.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

Número máximo de folhas é utilizado para limitar a quantidade de folhas, escolhendo apenas as melhores folhas. 

In [ ]:
# Mudança 3 - max_leaf_nodes=10
rf10 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700, max_leaf_nodes=10,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

rf10.fit(train[feats], train['Target'])

test['Target'] = rf10.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

# IV - Balanceamento das Classes


In [ ]:
# Importando a biblioteca
from sklearn.utils import resample
import scikitplot as skplt

# Separando os dados de acordo com a classificação
df_1 = df_all[df_all['Target'] == 1]
df_2 = df_all[df_all['Target'] == 2]
df_3 = df_all[df_all['Target'] == 3]
df_4 = df_all[df_all['Target'] == 4]

df_1.shape, df_2.shape, df_3.shape, df_4.shape

Random Over-Sampling
Aumentando a classe minoritária

In [ ]:
# Over-Sampling
df_1_over = resample(df_1, # vamos aumentar a classe menor
                       replace=True, # sample com replacement
                       n_samples=len(df_4), # igualando a maior classe
                       random_state=42)

# Over-Sampling
df_2_over = resample(df_2, # vamos aumentar a classe menor
                       replace=True, # sample com replacement
                       n_samples=len(df_4), # igualando a maior classe
                       random_state=42)

# Over-Sampling
df_3_over = resample(df_3, # vamos aumentar a classe menor
                       replace=True, # sample com replacement
                       n_samples=len(df_4), # igualando a maior classe
                       random_state=42)



# juntando os dados
df_over= pd.concat([df_1_over, df_2_over, df_3_over, df_4])

# check new class counts
df_over['Target'].value_counts()

In [ ]:
# Quais colunas do dataframe são do tipo object
df_over.select_dtypes('object').head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Executando o modelo com df_over

# Dividindo em treino e teste
train, test = train_test_split(df_over, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['Target'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['Target'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['Target'], preds_test)

Random Under-Sampling
Diminuindo a classe majoritária

In [ ]:
# Under-Sampling
df_2_under = resample(df_2, # vamos diminuir as classes maiores
                       replace=False, # sample sem replacement
                       n_samples=len(df_1), # igualando a menor classe
                       random_state=42)

# Under-Sampling
df_3_under = resample(df_3, # vamos aumentar a classe menor
                       replace=False, # sample se replacement
                       n_samples=len(df_1), # igualando a menor classe
                       random_state=42)
# Under-Sampling
df_4_under = resample(df_4, # vamos aumentar a classe menor
                       replace=False, # sample se replacement
                       n_samples=len(df_1), # igualando a menor classe
                       random_state=42)



# juntando os dados
df_under= pd.concat([df_1, df_2_under, df_3_under, df_4_under])

# check new class counts
df_under['Target'].value_counts()

In [ ]:
# Executando o modelo com df_under

# Dividindo em treino e teste
train, test = train_test_split(df_under, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['Target'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['Target'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['Target'], preds_test)

Usando a bilioteca imbalanced-learn
Essa biblioteca implementa diversos modelos diferentes para tratar classes desabalanceadas

Tive problemas com essa biblioteca:
ModuleNotFoundError: No module named 'sklearn.neighbors._base'

In [ ]:
# Importando a biblioteca
import imblearn

In [ ]:
# Separando os dados de entrada e o target
X, y = df_all[feats], df_all[['Target']]

Imblearn Random Over-Sampling

In [ ]:
# Importando a biblioteca
from imblearn.over_sampling import RandomOverSampler

# Fazendo o over-sampling
ros = RandomOverSampler(random_state=42)
X_ros,y_ros= ros.fit_resample(X,y)

# Verificando os dados
y_ros['Target'].value_counts()

In [ ]:
# Executando o modelo com imblearn over-sampling

# Juntando os dados
df_over = pd.concat([X_ros, y_ros], axis=1)

# Dividindo em treino e teste
train, test = train_test_split(df_over, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['Target'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['Target'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['Target'], preds_test)

Imblearn Tomek-links (under-sampling)

In [ ]:
# Importando a biblioteca
from imblearn.under_sampling import TomekLinks

# Fazendo o under-sampling
tl = TomekLinks()
X_tl, y_tl = tl.fit_resample(X,y)

# Verificando os dados
y_tl['Target'].value_counts()

In [ ]:
# Executando o modelo com Tomek-links

# Juntando os dados
df_under = pd.concat([X_tl, y_tl], axis=1)

# Dividindo em treino e teste
train, test = train_test_split(df_under, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['Target'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['Target'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['Target'], preds_test)

Imblearn SMOTE (over-sampling)

In [ ]:
# Importando a biblioteca
from imblearn.over_sampling import SMOTE

# Fazendo o under-sampling
sm = SMOTE()
X_sm, y_sm = sm.fit_resample(X,y)

# Verificando os dados
y_sm['Target'].value_counts()

In [ ]:
# Executando o modelo com SMOTE

# Juntando os dados
df_over = pd.concat([X_sm, y_sm], axis=1)

# Dividindo em treino e teste
train, test = train_test_split(df_over, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['Target'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['Target'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['Target'], preds_test)